#Import statements

In [307]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, dok_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from scipy.sparse import csr_matrix
from scipy.sparse import dok_matrix

#FIS

In [308]:
import numpy as np
import pandas as pd
import os

def fis_algorithm_with_auxiliary(X, num_iterations=200, learning_rate=0.001, reg_strength=0.001):
    """
    Apply the FIS algorithm to learn weights for binary matrix X with normalization and scaling.
    Takes a NumPy array or pandas DataFrame as input for X.
    """

    def initialize_weights(num_features):
        """Initialize weights to random values, instead of zeros."""
        return np.random.randn(num_features)  # Random initialization from a normal distribution

    def calculate_normalization(X, weights, clip_value=500):
        """Calculate row-wise normalization Z(x) and return a global normalization parameter."""
        exp_values = np.exp(X @ weights)  # Shape: (num_samples,)
        exp_values = np.clip(exp_values, None, clip_value)  # Clip to avoid overflow
        row_norms = exp_values  # Row-wise normalization since exp_values is already 1D
        global_norm = np.mean(row_norms)  # Aggregate to a scalar
        return global_norm, row_norms

    def calculate_log_likelihood(X, weights):
        """Calculate the log-likelihood using row-specific normalization."""
        global_norm, row_norms = calculate_normalization(X, weights)
        log_likelihood_per_row = X @ weights - np.log(row_norms)
        total_log_likelihood = np.sum(log_likelihood_per_row)  # Sum over rows
        return total_log_likelihood, global_norm

    def auxiliary_function(X, weights, delta):
        """Auxiliary function Q(δ) derived from the change in log-likelihood."""
        observed_support = np.sum(X, axis=0)
        global_norm, row_norms = calculate_normalization(X, weights)
        expected_support = np.sum(np.exp(X @ (weights + delta)) / row_norms, axis=0)
        Q = np.sum(delta * observed_support - expected_support)
        return Q

    def update_weights_using_auxiliary(X, weights, learning_rate=0.001, reg_strength=0.001, max_weight_value=100):
        """Update weights based on the auxiliary function and row-specific normalization."""
        num_features = X.shape[1]
        delta = np.zeros(num_features)
        global_norm, row_norms = calculate_normalization(X, weights)
        observed_support = np.sum(X, axis=0)

        for i in range(num_features):
            expected_support = np.sum(np.exp(X[:, i] * weights[i]) / row_norms)  # Row-specific normalization
            delta[i] = observed_support[i] - expected_support

        weights += learning_rate * (delta - reg_strength * weights)
        weights = np.clip(weights, -max_weight_value, max_weight_value)
        return weights, global_norm

    def scale_to_01(values):
        """Scale the input values to the range [0, 1]."""
        min_val = np.min(values)
        max_val = np.max(values)
        return (values - min_val) / (max_val - min_val) if max_val > min_val else values

    # Ensure X is a NumPy array
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()  # Convert DataFrame to NumPy array

    # Initialize weights
    num_features = X.shape[1]
    weights = initialize_weights(num_features)
    w_norm = None  # Track the global normalization parameter

    # Apply the FIS algorithm to the matrix X
    for iteration in range(num_iterations):
        # Calculate log-likelihood and normalization
        log_likelihood, w_norm = calculate_log_likelihood(X, weights)

        # Scale the log-likelihood to [0, 1]
        log_likelihood_scaled = scale_to_01(np.array([log_likelihood]))[0]

        print(f"Iteration {iteration}, Log-Likelihood (scaled): {log_likelihood_scaled}")

        # Update weights using auxiliary function and normalization
        weights, w_norm = update_weights_using_auxiliary(X, weights, learning_rate, reg_strength)

        # Scale weights to [0, 1]
        weights = scale_to_01(weights)

    print("Learned Weights (scaled):", weights)
    print("Final Normalization Parameter (w_norm):", w_norm)

    return weights, w_norm

#Categorize->Dice->intervals->missing->binarize->transform->filter

In [309]:
def categorize_column(dataset, dependent_column_name, num_intervals):
    # Fill NaN values with the mean of the column
    dataset_filled = dataset.fillna(dataset[dependent_column_name].mean())

    # Extract the specified dependent column
    value_ag_filled = dataset_filled[dependent_column_name]

    # Define the target range (0-1)
    target_range = (0, 1)

    # Scale to the target range [0, 1] using np.interp
    value_ag_scaled_filled = np.interp(value_ag_filled, (value_ag_filled.min(), value_ag_filled.max()), target_range)

    # Create intervals and assign labels for scaled values
    data_inter_filled = pd.cut(value_ag_scaled_filled, bins=num_intervals, labels=[chr(ord('A') + i) for i in range(num_intervals)])

    # Store intervals in a DataFrame
    result_df = pd.DataFrame({
        'Category': data_inter_filled,
        dependent_column_name: value_ag_scaled_filled
    })

    return result_df

import pandas as pd
import os
import numpy as np

# Define the path to your Temporary_SXmat directory where X_interval files are saved
save_path = '/content/drive/MyDrive/Temporary_SXmat'

# Load your frequent_patterns dataframe (replace with your actual frequent patterns dataframe)
# For example, assuming `support_df` is loaded from a CSV
# support_df = pd.read_csv('path_to_support_patterns.csv')

def calculate_dice_similarity(dataset_df, support_df):
    """
    Calculate the Dice-Sørensen Coefficient (DSC) matrix for a given dataset and frequent patterns.

    Args:
        dataset_df (pd.DataFrame): The dataset dataframe.
        support_df (pd.DataFrame): The frequent patterns dataframe (support patterns).

    Returns:
        pd.DataFrame: A dataframe containing the DSC matrix.
    """
    unique_items = set()
    for pattern in support_df.iloc[:, 0]:  # Assuming patterns are in the first column
        pattern = str(pattern)
        if pd.notna(pattern):
            try:
                items = pattern.split(', ')
                unique_items.update(items)
            except AttributeError:
                print(f"Skipping pattern {pattern} as it's not a valid string.")

    unique_item_mapping = {item: idx for idx, item in enumerate(unique_items)}
    bit_vector_length = len(unique_item_mapping)

    # Convert dataset rows to bit vectors
    def row_to_bit_vector(row, item_mapping, bit_length):
        bit_vector = np.zeros(bit_length, dtype=int)
        for col, value in row.items():
            item = f"{col}={value}"
            if item in item_mapping:
                bit_vector[item_mapping[item]] = 1
        return bit_vector

    dataset_bit_vectors = np.array([row_to_bit_vector(row, unique_item_mapping, bit_vector_length) for _, row in dataset_df.iterrows()])

    # Convert support patterns to bit vectors
    def pattern_to_bit_vector(pattern, item_mapping, bit_length):
        bit_vector = np.zeros(bit_length, dtype=int)
        try:
            pattern = str(pattern)
            for item in pattern.split(', '):
                if item in item_mapping:
                    bit_vector[item_mapping[item]] = 1
        except AttributeError:
            print(f"Skipping pattern {pattern} as it's not a valid string.")
        return bit_vector

    support_pattern_bit_vectors = np.array([
        pattern_to_bit_vector(pattern, unique_item_mapping, bit_vector_length)
        for pattern in support_df.iloc[:, 0]  # Assuming patterns are in the first column
        if pd.notna(pattern)
    ])

    # Initialize the DSC matrix with the correct shape
    dsc_matrix = np.zeros((len(dataset_bit_vectors), len(support_pattern_bit_vectors)))

    # Calculate DSC for each combination of dataset row and support pattern
    for i, data_vector in enumerate(dataset_bit_vectors):
        for j, pattern_vector in enumerate(support_pattern_bit_vectors):
            intersection = np.sum(data_vector & pattern_vector)
            union_cardinality = np.sum(data_vector) + np.sum(pattern_vector)
            if union_cardinality > 0:
                dsc_matrix[i, j] = (2 * intersection) / union_cardinality
            else:
                dsc_matrix[i, j] = 0

    # Create a dataframe with the DSC matrix
    dsc_df = pd.DataFrame(dsc_matrix, columns=[f'Pattern_{k}' for k in range(len(support_pattern_bit_vectors))])
    return dsc_df

def create_intervals_df(data_frame, column_name, num_intervals):
    # Extract the specified column
    value_ag = data_frame[column_name]

    #print("Extracted column values:")
    #print(value_ag)

    # Check if the column contains numeric data
    if not pd.api.types.is_numeric_dtype(value_ag):
        raise ValueError(f"The '{column_name}' column must contain numeric data.")

    # Check for missing values in the column
    if value_ag.isnull().any():
        raise ValueError(f"The '{column_name}' column contains missing values. Please handle them before processing.")

    # Check if scaling and clipping is necessary
    if value_ag.min() < 0 or value_ag.max() > 1:
        # Rescale to the range [0, 1]
        value_ag_scaled = (value_ag - value_ag.min()) / (value_ag.max() - value_ag.min())

        # Clip values to the range [0, 1]
        value_ag_scaled_clipped = np.clip(value_ag_scaled, 0, 1)
    else:
        # No need to rescale or clip
        value_ag_scaled_clipped = value_ag

    #print("Scaled and clipped column values:")
    #print(value_ag_scaled_clipped)

    # Calculate bin edges dynamically within the [0, 1] range
    bin_edges = np.linspace(0, 1, num=num_intervals + 1)

    # Create intervals and assign labels for clipped values
    data_inter_clipped = pd.cut(value_ag_scaled_clipped.values, bins=bin_edges, labels=[chr(ord('A') + i) for i in range(len(bin_edges) - 1)])

    #print("Computed intervals:")
    #print(data_inter_clipped)

    # Store intervals in a DataFrame
    intervals_df = pd.DataFrame({
        'Category': data_inter_clipped.categories,
        'Min_Value': [value_ag_scaled_clipped[data_inter_clipped == category].min() for category in data_inter_clipped.categories],
        'Max_Value': [value_ag_scaled_clipped[data_inter_clipped == category].max() for category in data_inter_clipped.categories]
    })

    return intervals_df



def handle_missing_values(dataset):
    # Check if there are missing values in the dataset
    if dataset.isnull().any().any():
        # Identify numerical and categorical columns
        numerical_cols = dataset.select_dtypes(include=['number']).columns
        categorical_cols = dataset.select_dtypes(exclude=['number']).columns

        # Impute numerical columns with mean
        imputer_numeric = SimpleImputer(strategy='mean')
        dataset[numerical_cols] = imputer_numeric.fit_transform(dataset[numerical_cols])

        # Impute categorical columns with the most frequent value (mode)
        imputer_categorical = SimpleImputer(strategy='most_frequent')
        dataset[categorical_cols] = imputer_categorical.fit_transform(dataset[categorical_cols])

        return dataset
    else:
        # If no missing values, return the original dataset
        return dataset

def binarize_dataframe(input_df, num_bins):
    df = pd.DataFrame(input_df)
    df_binarized = pd.DataFrame()

    for column in df.columns:
        column_name = column+'_binarized'
        bins = pd.qcut(df[column], q=num_bins, labels=False, duplicates='drop')
        df_binarized[column_name] = (bins == bins.max()).astype(int)

    return df_binarized

def transform_matrix_dice(matrix):
    # Create a copy of the input matrix
    transformed_matrix = matrix.copy()
    print(transformed_matrix)
    # Apply the transformation
    transformed_matrix[transformed_matrix > 0.40] = 1
    transformed_matrix[transformed_matrix <= 0.40] = 0
    return transformed_matrix

def transform_matrix_dice(matrix):
    # Create a copy of the input matrix
    transformed_matrix = matrix.copy()
    #print("Inside Transformed")
    # Apply the transformation
    transformed_matrix[transformed_matrix > 0.40] = 1
    transformed_matrix[transformed_matrix <= 0.40] = 0
    return transformed_matrix


def filter_data_for_interval(df, category_col, category):
    # Filter data for a specific interval category
    return df[df[category_col] == category]

def transform_matrix_for_interval(X_interval, support_df):
    jaccard_result = calculate_dice_similarity(X_interval, support_df)
    #print("Inside Transform")
    #print(jaccard_result.shape)
    transformed_matrix = transform_matrix_dice(jaccard_result)
    print(transformed_matrix.shape)
    return transformed_matrix

#Bayesian model for FP(with interestingness)

In [310]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Binarizer
from scipy.special import logsumexp

def find_separate_itemsets_for_measures(dataset_path, freq_pattern_path, drop_column=None, max_iterations=100, convergence_threshold=1e-4, thresholds=None):
    # Load dataset and frequent patterns
    dataset = pd.read_csv(dataset_path)
    freq_patterns = pd.read_csv(freq_pattern_path, sep='\t')

    # Default thresholds if not provided
    if thresholds is None:
        thresholds = {
            "support": 0.01,
            "confidence": 0.000000000001,
            "lift": 0.01,
            "leverage": 0.0,
            "jaccard": 0.5,
            "cosine": 0.5
        }

    # Preprocess dataset - drop column if specified
    if drop_column:
        X = dataset.drop(columns=[drop_column])
    else:
        X = dataset  # If no column is specified to drop, use the full dataset

    binary_data = Binarizer().fit_transform(X)

    # Initialize itemset probabilities pi_s for each pattern
    pi = np.ones(len(freq_patterns))  # Initialize all itemsets with probability 1 (uniform prior)

    # Helper functions for Bayesian Mixture Model

    def e_step(data, pi):
        epsilon = 1e-10  # Small value to avoid log of zero
        log_resp = np.zeros((data.shape[0], len(pi)))  # Initialize the responsibilities
        for i, pattern in enumerate(freq_patterns['Itemsets']):
            # Parse the pattern to get item columns
            pattern_items = [item.split('=')[0] for item in pattern.split()]
            pattern_columns = [col for col in X.columns if col in pattern_items]
            if not pattern_columns:
                continue  # Skip patterns that don't match any columns

            # Indicator function: 1 if itemset is present, 0 if not
            si = np.zeros(data.shape[0])
            for j, row in enumerate(data):
                si[j] = 1 if all(row[X.columns.get_loc(col)] for col in pattern_columns) else 0

            # Calculate log-probability of the data under each itemset
            log_prob = si * np.log(pi[i] + epsilon) + (1 - si) * np.log(1 - pi[i] + epsilon)
            log_resp[:, i] = log_prob

        # Normalize the responsibilities across all itemsets (log-sum-exp trick)
        log_resp -= logsumexp(log_resp, axis=1)[:, np.newaxis]
        return np.exp(log_resp)

    def m_step(data, responsibilities):
        N, D = data.shape
        Nk = responsibilities.sum(axis=0)
        pi_new = Nk / N  # Update component priors (probabilities of each itemset)
        return pi_new

    # Run EM algorithm until convergence or maximum iterations
    for iteration in range(max_iterations):
        prev_pi = pi.copy()

        # Perform E-step and M-step
        responsibilities = e_step(binary_data, pi)
        pi = m_step(binary_data, responsibilities)

        # Check for convergence
        delta_pi = np.linalg.norm(pi - prev_pi)
        if delta_pi < convergence_threshold:
            print(f"Converged after {iteration + 1} iterations.")
            break
    else:
        print("Reached maximum iterations without convergence.")

    # Initialize separate results for each interestingness measure
    support_results = []
    confidence_results = []
    lift_results = []
    leverage_results = []
    jaccard_results = []
    cosine_results = []

    # Calculate interestingness measures for each pattern
    for _, row in freq_patterns.iterrows():
        pattern_string = row['Itemsets']

        # Parse the pattern string into column names and values
        pattern_items = [
            item.strip().split('=')[0]  # Extract the attribute (column name)
            for item in pattern_string.split()  # Split by spaces
            if '=' in item  # Ignore parts without '=' (e.g., "count=8")
        ]

        # Match column names in the dataset
        pattern_columns = [col for col in X.columns if col in pattern_items]
        if not pattern_columns:
            continue  # Skip patterns with no matching columns

        # Calculate support
        support_count = (binary_data[:, [X.columns.get_loc(col) for col in pattern_columns]].sum(axis=1)
                         == len(pattern_columns)).sum()
        support = support_count / binary_data.shape[0]

        # Confidence (Assuming a simple rule A -> B)
        if len(pattern_columns) > 1:
            A, B = pattern_columns[:-1], pattern_columns[-1]
            support_A = (binary_data[:, [X.columns.get_loc(col) for col in A]].sum(axis=1) == len(A)).sum() / binary_data.shape[0]
            confidence = support / (support_A + 1e-10)
        else:
            confidence = np.nan

        # Lift
        if len(pattern_columns) > 1:
            support_B = (binary_data[:, X.columns.get_loc(B)] == 1).sum() / binary_data.shape[0]
            lift = support / (support_A * support_B + 1e-10)
        else:
            lift = np.nan

        # Leverage
        leverage = support - (support_A * support_B) if len(pattern_columns) > 1 else np.nan

        # Jaccard Index
        jaccard = support / (support_A + support_B - support) if len(pattern_columns) > 1 else np.nan

        # Cosine Similarity
        cosine = support / (support_A * support_B + 1e-10) if len(pattern_columns) > 1 else np.nan

        # Store results if they exceed the threshold
        if support >= thresholds['support']:
            support_results.append({"Pattern": pattern_string, "Support": support})
        if confidence >= thresholds['confidence']:
            confidence_results.append({"Pattern": pattern_string, "Confidence": confidence})
        if lift >= thresholds['lift']:
            lift_results.append({"Pattern": pattern_string, "Lift": lift})
        if leverage >= thresholds['leverage']:
            leverage_results.append({"Pattern": pattern_string, "Leverage": leverage})
        if jaccard >= thresholds['jaccard']:
            jaccard_results.append({"Pattern": pattern_string, "Jaccard": jaccard})
        if cosine >= thresholds['cosine']:
            cosine_results.append({"Pattern": pattern_string, "Cosine": cosine})

    # Convert results to DataFrames
    support_df = pd.DataFrame(support_results)
    confidence_df = pd.DataFrame(confidence_results)
    lift_df = pd.DataFrame(lift_results)
    leverage_df = pd.DataFrame(leverage_results)
    jaccard_df = pd.DataFrame(jaccard_results)
    cosine_df = pd.DataFrame(cosine_results)

    return support_df, confidence_df, lift_df, leverage_df, jaccard_df, cosine_df

#Density->process_interval

In [311]:
def calculate_density(matrix):
    num_ones = matrix.nnz if isinstance(matrix, csr_matrix) else np.count_nonzero(matrix)
    total_elements = matrix.shape[0] * matrix.shape[1]
    density = num_ones / total_elements
    return density

# def process_interval(row):
#     category = row['Category']
#     interval_min = row['Min_Value']
#     interval_max = row['Max_Value']

#     filtered_data = filter_data_for_interval(Data_discretized, 'Category', category)
#     if not filtered_data.empty:
#         X_interval = filtered_data.drop(columns=['Category'])
#         print(X_interval.shape)
#         val = calculate_dice_similarity(X_interval, support_df)
#         #print(val.shape)
#         val1 = transform_matrix_for_interval(X_interval, support_df)
#         SXmat = val1
#         SXmat = pd.DataFrame(SXmat)
#         # Calculate Density
#         density = calculate_density(SXmat)
#         print(f"Density of incidence matrix for this interval (Category {category}): {density}")
#         weights, w_norm = fis_algorithm_with_auxiliary(SXmat)
#         return category, {'weights': weights, 'wnorm': w_norm}
#     else:
#         print(f"No data for interval {category}")
#         return category, None

def process_interval(row):
    category = row['Category']
    interval_min = row['Min_Value']
    interval_max = row['Max_Value']

    filtered_data = filter_data_for_interval(Data_discretized, 'Category', category)
    if not filtered_data.empty:
        X_interval = filtered_data.drop(columns=['Category'])
        print(X_interval.shape)
        val = calculate_dice_similarity_demo(X_interval, support_df)
        print(val.shape)
        val1 = transform_matrix_for_interval(X_interval, support_df)
        SXmat = val1
        SXmat = pd.DataFrame(SXmat)
        print(SXmat.head(1))
        df_SXmat = pd.DataFrame(SXmat)
        sxmat_file_path = os.path.join(save_path, f'SXmat_Category_{category}.csv')
        df_SXmat.to_csv(sxmat_file_path, index=False)
        print(f"SXmat for Category {category} saved at {sxmat_file_path}")
        # Calculate Density
        density = calculate_density(SXmat)
        print(f"Density of incidence matrix for this interval (Category {category}): {density}")
        weights, w_norm = fis_algorithm_with_auxiliary(SXmat)
        print(weights.shape)
        #print(wnorm.shape)
        return category, {'weights': weights, 'wnorm': w_norm}
    else:
        print(f"No data for interval {category}")
        return category, None



#Process_test_dataset

In [312]:
def process_test_data(Data_test, support_df, categories_weights, constant):
    # Create a temporary DataFrame as a copy of Data_test
    Data_test_temp = Data_test.copy()

    # Iterate through each row in Data_test_temp
    for index, row in Data_test_temp.iterrows():
        # Convert the row to a DataFrame for compatibility with transform_matrix_for_interval
        row_df = pd.DataFrame([row])

        # Calculate the Jaccard/Dice similarity for the current row with support_df
        jaccard_result = calculate_dice_similarity_demo(row_df, support_df)

        # Directly apply binarization with a threshold of 0.4
        transformed_row = jaccard_result.copy()
        transformed_row[transformed_row > 0.4] = 1
        transformed_row[transformed_row <= 0.4] = 0

        # Flatten the transformed_row to ensure it is 1D (1, 87) -> (87,)
        transformed_row_flat = transformed_row.values.flatten()

        max_value = 0.0  # Initialize the max value
        predicted_category = None  # Store the category with the highest value

        # Iterate over each category and calculate the weighted product
        for category, params in categories_weights.items():
            # Ensure the structure is valid
            if 'weights' in params and 'wnorm' in params:
                weights = params['weights']  # Extract weights for the category
                wnorm = params['wnorm']      # Extract normalization value

                # Reshape weights to (1, 87) to match the transformed_row shape (1, 87)
                weights_reshaped = weights.reshape(1, -1)

                # Ensure both weights and transformed_row are compatible (both should be (1, 87))
                if weights_reshaped.shape != transformed_row.shape:
                    print(f"Shape mismatch between weights and transformed_row for category {category}")
                    continue

                product = weights_reshaped * transformed_row  # Element-wise multiplication
                # Sum the product over all columns (axis=1 sums across columns)
                product_sum = product.sum(axis=1)  # This will give a single value
                # Calculate the final product with the normalization value and constant
                final_product = (product_sum + wnorm) * constant
                final_product = final_product.item()
                # Check if the current product is higher than max_value
                if final_product > max_value:
                    max_value = final_product
                    predicted_category = category  # Store the category with the highest product

        # After processing all categories for the current row, add the predicted category to Data_test_temp
        Data_test_temp.loc[index, 'predicted_category'] = predicted_category

    # Return the updated DataFrame with the predicted category column
    return Data_test_temp

#RMSE calculation

In [326]:
import numpy as np

def calculate_rmse_category(X_test, intervals_df, updated_data):
    sum_squared_diff = 0.0
    sum_absolute_diff = 0.0
    actual_values = []
    predicted_values = []

    # Iterate through the test dataset to calculate RMSE, MAE, and collect values for PCC
    for i, row in X_test.iterrows():
        actual_category = row['Category']  # Actual category
        predicted_category = updated_data.loc[i, 'predicted_category']  # Predicted category from updated_data
        #print(f"Predicted Category: {predicted_category}")

        # Get the actual 'Max_Value' based on the actual category
        actual_max_value = intervals_df[intervals_df['Category'] == actual_category]['Max_Value'].values

        # Check if predicted_category exists in intervals_df
        if predicted_category in intervals_df['Category'].values:
            predicted_max_value = intervals_df[intervals_df['Category'] == predicted_category]['Max_Value'].values
        else:
            # Handle case where predicted category is not found in intervals_df
            #print(f"Predicted category '{predicted_category}' not found in intervals_df.")
            predicted_max_value = np.nan  # Set predicted_max_value to NaN or handle it as needed

        #print(f"Actual Max Value: {actual_max_value}, Predicted Max Value: {predicted_max_value}")

        # Ensure the actual max value exists and predicted max value is not NaN
        if len(actual_max_value) > 0:
            actual_max_value = actual_max_value[0]

            # Handle NaN values in actual_max_value or predicted_max_value
            if np.isnan(actual_max_value) or np.isnan(predicted_max_value):
                #print(f"Skipping pair due to NaN value (Actual: {actual_max_value}, Predicted: {predicted_max_value})")

                # Option 1: Set difference to 0 if both are NaN
                squared_diff = 0
                absolute_diff = 0
            else:
                # Calculate squared difference for RMSE
                squared_diff = (predicted_max_value - actual_max_value) ** 2
                absolute_diff = abs(predicted_max_value - actual_max_value)

            sum_squared_diff += squared_diff
            sum_absolute_diff += absolute_diff

            # Collect values for PCC calculation
            actual_values.append(actual_max_value)
            predicted_values.append(predicted_max_value)

    # Ensure we have data to calculate the metrics
    if len(X_test) > 0:
        # Calculate RMSE
        mean_squared_error = sum_squared_diff / len(X_test)
        rmse = np.sqrt(mean_squared_error).item()  # Convert numpy array to scalar

        # Calculate MAE
        mae = (sum_absolute_diff / len(X_test)).item()  # Convert numpy array to scalar

        return rmse, mae

    # Return None if there are no rows in X_test
    return None, None


#Check dataset

In [314]:
import pandas as pd
# Load Dataset while skipping the 'datetime' column
dataset = "/content/drive/MyDrive/concrete_updated.csv"
concrete = pd.read_csv(dataset)
concrete.head(2)

,No,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm),FLOW(cm),Strength
0,1,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0,62.0,34.99
1,2,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0,20.0,41.14


#Calling the function with dataset input

In [318]:
import pandas as pd
# Load Dataset while skipping the 'datetime' column
dataset = "/content/drive/MyDrive/bolts.csv"
concrete = pd.read_csv(dataset)
dependent = "T20BOLT"
concrete_temp = concrete.copy()
X = concrete_temp.drop(dependent, axis=1).copy()
y = concrete[dependent]
# Usage example:
dataset_path = "/content/drive/MyDrive/bolts.csv"
freq_pattern_path = '/content/drive/MyDrive/AprioriOutput/closed_itemsets_bolts.csv'
drop_column = 'T20BOLT'  # Example of the column to drop
support_df, confidence_df, lift_df, leverage_df, jaccard_df, cosine_df = find_separate_itemsets_for_measures(dataset_path, freq_pattern_path, drop_column=drop_column)
support_df = support_df.drop(columns=['Support'])
# Handle Missing Values
X = handle_missing_values(X)
# Binarize DataFrame
num_bins = 15
df_binarized = binarize_dataframe(pd.DataFrame(concrete), num_bins)
# Create Intervals DataFrame
num_intervals = 4
intervals_df = create_intervals_df(concrete, dependent, num_intervals)
print(intervals_df)
# Categorize Column
result_df = categorize_column(concrete, dependent, num_intervals)
P_c = 1 / num_intervals
P_x = 1 / concrete.shape[1]
constant = P_c / P_x
Data_discretized = pd.read_csv('/content/drive/MyDrive/Discretized_datasets/discretized_bolts.csv')
Data_discretized['Category'] = result_df['Category']

Converged after 15 iterations.
  Category  Min_Value  Max_Value
0        A   0.006828   0.237991
1        B   0.317240   0.338942
2        C   0.615216   0.746159
3        D   0.776030   1.000000


In [319]:
# Calculate Jaccard Similarity and Train GIS for Each Interval
categories_weights = {}
categories = Data_discretized['Category'].unique()
# Use ThreadPoolExecutor to run computations in parallel
with ThreadPoolExecutor() as executor:
    # Create a list of futures
    futures = {executor.submit(process_interval, row): row for _, row in intervals_df.iterrows()}
    # Process completed futures
    for future in concurrent.futures.as_completed(futures):
        category, weights_wnorm = future.result()
        if weights_wnorm is not None:
            categories_weights[category] = weights_wnorm

(3, 7)
(26, 7)
(3, 7)
(9, 7)
(3, 199)(3, 199)

(3, 199)
(9, 199)(3, 199)
   Pattern_0  Pattern_1  Pattern_2  Pattern_3  Pattern_4  Pattern_5  \
0        0.0        0.0        0.0        0.0        0.0        0.0   

   Pattern_6  Pattern_7  Pattern_8  Pattern_9  ...  Pattern_189  Pattern_190  \
0        0.0        0.0        0.0        0.0  ...          0.0          0.0   

   Pattern_191  Pattern_192  Pattern_193  Pattern_194  Pattern_195  \
0          0.0          0.0          0.0          0.0          0.0   

   Pattern_196  Pattern_197  Pattern_198  
0          0.0          0.0          0.0  

[1 rows x 199 columns]
   Pattern_0  Pattern_1  Pattern_2  Pattern_3  Pattern_4  Pattern_5  \
0        0.0        0.0        0.0        0.0        0.0        0.0   

   Pattern_6  Pattern_7  Pattern_8  Pattern_9  ...  Pattern_189  Pattern_190  \
0        0.0        0.0        0.0        0.0  ...          1.0          0.0   

   Pattern_191  Pattern_192  Pattern_193  Pattern_194  Pattern_195  

<ipython-input-308-527064c62d59>:26: RuntimeWarning: divide by zero encountered in log
  log_likelihood_per_row = X @ weights - np.log(row_norms)
<ipython-input-308-527064c62d59>:46: RuntimeWarning: divide by zero encountered in divide
  expected_support = np.sum(np.exp(X[:, i] * weights[i]) / row_norms)  # Row-specific normalization


Iteration 29, Log-Likelihood (scaled): 61.471552954740226
Iteration 4, Log-Likelihood (scaled): inf
Iteration 27, Log-Likelihood (scaled): 100.29738646453049
Iteration 30, Log-Likelihood (scaled): 61.61311468017795

Iteration 5, Log-Likelihood (scaled): inf
Iteration 31, Log-Likelihood (scaled): 61.685619999219604

Iteration 28, Log-Likelihood (scaled): 100.15237174871632Iteration 6, Log-Likelihood (scaled): inf
Iteration 29, Log-Likelihood (scaled): 100.00764583251285Iteration 32, Log-Likelihood (scaled): 61.7579804931359
Iteration 30, Log-Likelihood (scaled): 99.8632081395552
Iteration 33, Log-Likelihood (scaled): 61.83019645034709
Iteration 7, Log-Likelihood (scaled): inf
Iteration 34, Log-Likelihood (scaled): 61.902268158701126
Iteration 31, Log-Likelihood (scaled): 99.71905809463556
Iteration 8, Log-Likelihood (scaled): inf
Iteration 32, Log-Likelihood (scaled): 99.57519512370064
Iteration 35, Log-Likelihood (scaled): 61.97419590547497

Iteration 9, Log-Likelihood (scaled): inf

I

<ipython-input-308-527064c62d59>:46: RuntimeWarning: divide by zero encountered in divide
  expected_support = np.sum(np.exp(X[:, i] * weights[i]) / row_norms)  # Row-specific normalization
<ipython-input-308-527064c62d59>:26: RuntimeWarning: divide by zero encountered in log
  log_likelihood_per_row = X @ weights - np.log(row_norms)
<ipython-input-308-527064c62d59>:46: RuntimeWarning: divide by zero encountered in divide
  expected_support = np.sum(np.exp(X[:, i] * weights[i]) / row_norms)  # Row-specific normalization
<ipython-input-308-527064c62d59>:26: RuntimeWarning: divide by zero encountered in log
  log_likelihood_per_row = X @ weights - np.log(row_norms)
<ipython-input-308-527064c62d59>:46: RuntimeWarning: divide by zero encountered in divide
  expected_support = np.sum(np.exp(X[:, i] * weights[i]) / row_norms)  # Row-specific normalization
<ipython-input-308-527064c62d59>:26: RuntimeWarning: divide by zero encountered in log
  log_likelihood_per_row = X @ weights - np.log(row

Iteration 55, Log-Likelihood (scaled): 96.34427596136743
Iteration 56, Log-Likelihood (scaled): 96.207133510987
Iteration 57, Log-Likelihood (scaled): 96.07026415168602
Iteration 58, Log-Likelihood (scaled): 95.93366733861713
Iteration 80, Log-Likelihood (scaled): 65.06649347047994
Iteration 81, Log-Likelihood (scaled): 65.13209988848794
Iteration 48, Log-Likelihood (scaled): inf
Iteration 82, Log-Likelihood (scaled): 65.19757522308961
Iteration 83, Log-Likelihood (scaled): 65.26291973543782
Iteration 84, Log-Likelihood (scaled): 65.32813368616709
Iteration 85, Log-Likelihood (scaled): 65.39321733539482
Iteration 86, Log-Likelihood (scaled): 65.45817094272208
Iteration 87, Log-Likelihood (scaled): 65.52299476723472
Iteration 59, Log-Likelihood (scaled): 95.7973425280256
Iteration 60, Log-Likelihood (scaled): 95.6612891772472
Iteration 88, Log-Likelihood (scaled): 65.5876890675044
Iteration 49, Log-Likelihood (scaled): inf
Iteration 50, Log-Likelihood (scaled): inf
Iteration 89, Log-Lik

In [325]:
# Split the data into training and testing datasets for model evaluation
Data_train, Data_test = train_test_split(Data_discretized, test_size=0.2, random_state=5])
# Example of calling the function
updated_data = process_test_data(Data_test, support_df, categories_weights, constant)
# Calculate the root mean squared error (RMSE) for the test data
rmse = calculate_rmse_category(Data_test,intervals_df,updated_data)  # Use a custom RMSE calculation function
print("RMSE:", rmse)

Predicted Category: C
Actual Max Value: [0.74615947], Predicted Max Value: [0.74615947]
Predicted Category: B
Actual Max Value: [0.33894172], Predicted Max Value: [0.33894172]
Predicted Category: C
Actual Max Value: [1.], Predicted Max Value: [0.74615947]
Predicted Category: B
Actual Max Value: [0.23799073], Predicted Max Value: [0.33894172]
Predicted Category: C
Actual Max Value: [0.23799073], Predicted Max Value: [0.74615947]
Predicted Category: B
Actual Max Value: [0.23799073], Predicted Max Value: [0.33894172]
Predicted Category: B
Actual Max Value: [0.33894172], Predicted Max Value: [0.33894172]
Predicted Category: C
Actual Max Value: [0.23799073], Predicted Max Value: [0.74615947]
Predicted Category: C
Actual Max Value: [0.74615947], Predicted Max Value: [0.74615947]
RMSE: (0.2584759073329495, 0.16356444227695138)


In [328]:
from sklearn.model_selection import train_test_split

def find_best_rmse_random_state(Data_discretized, intervals_df, support_df, categories_weights, constant):
    best_random_state = None
    min_rmse = float('inf')  # Initialize with a large value to ensure any RMSE is smaller

    # Iterate through random states from 5 to 200
    for random_state in range(5, 201):
        # Split the data into training and testing datasets
        Data_train, Data_test = train_test_split(Data_discretized, test_size=0.2, random_state=random_state)

        # Process the test data
        updated_data = process_test_data(Data_test, support_df, categories_weights, constant)

        # Calculate the root mean squared error (RMSE) and MAE
        rmse, mae = calculate_rmse_category(Data_test, intervals_df, updated_data)

        # Compare only the first value (RMSE) from the tuple
        if rmse is not None and rmse < min_rmse:
            min_rmse = rmse
            best_random_state = random_state

    return best_random_state, min_rmse

# Example usage
best_random_state, min_rmse = find_best_rmse_random_state(Data_discretized, intervals_df, support_df, categories_weights, constant)
print(f"Best Random State: {best_random_state}, RMSE: {min_rmse}")


Best Random State: 43, RMSE: 0.1612690803690874
